In [14]:
from quantopian.research import returns, symbols
from quantopian.pipeline.experimental import QTradableStocksUS
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.filters import Q500US
from quantopian.research import run_pipeline, prices
from quantopian.pipeline import Pipeline


# Select a time range to inspect
start = '2008-01-01'
end = '2017-12-31'

bench_returns = returns(
    assets=symbols('SPY'),
    start=start,
    end=end,
)


def make_pipeline():
    base_universe = QTradableStocksUS()
    close_price = USEquityPricing.close.latest
    #profitable = mstar.valuation_ratios.ev_to_ebitda > 0  #TODO: incorproate this later
    #sentiment_score = SimpleMovingAverage(
    #    inputs=[stocktwits.bull_minus_bear],
    #    window_length=3,) #TODO: incorporate this later

    

    return Pipeline(
        columns={
            'close_price': close_price,
            #'sentiment_score': sentiment_score,
        },
        screen=Q500US()
    )

In [15]:
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date=start,
    end_date=end
)


tickers_list = pipeline_output.index.levels[1].unique()
all_t = pipeline_output.index.levels[0].unique()
all_p = prices(tickers_list, start=start, end=end)

In [ ]:
class TradingRRL(object):
    def __init__(self, T=1000, thisT = 1000, M=300, thisM = 300, N=0, init_t=10000, mu=10000, sigma=0.04, rho=1.0, n_epoch=10):
        self.T = T
        self.thisT = thisT
        self.M = M
        self.thisM = thisM
        self.N = N
        self.TOP = 20
        self.threshold = 0.0
        self.init_t = init_t
        self.mu = mu
        self.sigma = sigma
        self.rho = rho
        self.all_t = None
        self.all_p = None
        self.t = None
        self.p = None
        self.bench = None
        self.r = None
        self.x = np.zeros([T, M + 2])
        self.F = np.zeros((T + 1, N))
        self.FS = np.zeros((T + 1, N))
        self.R = np.zeros((T, N))
        self.w = np.ones((M + 2, N))
        self.w_opt = np.ones((M + 2, N))
        self.epoch_S = pd.DataFrame()
        self.n_epoch = n_epoch
        self.progress_period = 100
        self.q_threshold = 0.5
        self.b = np.ones((T+1, N))
        self.total = None
        self.bench = None
        self.tickers_list = None
        self.ticker_data = collections.defaultdict(dict)

    def quant(self, f):
        fc = f.copy()
        fc[np.where(np.abs(fc) < self.q_threshold)] = 0
        #return np.sign(fc)
        return fc

    def softmax(self, x):
        l2_norm = np.sqrt(x*x).sum()
        return x/l2_norm
        #e_x = np.exp(x)
        #return e_x / e_x.sum()

    def set_t_p_r(self, train_phase=True):
        if train_phase:
            self.t = self.all_t[self.init_t:self.init_t + self.T + self.M + 1]
            self.p = self.all_p[self.init_t:self.init_t + self.T + self.M + 1,:] ## TODO: add column dimension for assets > 1
            print('p dimension', self.p.shape)
            #self.r = -np.diff(self.p, axis=0)
            firstr = np.zeros((1, self.p.shape[1]))
            self.r = np.diff(self.p, axis=0)/self.p[:-1]
            self.r = np.concatenate((firstr, self.r), axis=0)
            print('r dimension', self.r.shape)
            pd.DataFrame(self.r).to_csv("smallr.csv", header=False, index=False)
        else:
            self.t = self.all_t[self.init_t:self.init_t + self.thisT + self.thisM + 1]
            self.p = self.all_p[self.init_t:self.init_t + self.thisT + self.thisM + 1,:]  ## TODO: add column dimension for assets > 1
            print('p dimension', self.p.shape)
            # self.r = -np.diff(self.p, axis=0)
            firstr = np.zeros((1, self.p.shape[1]))
            self.r = np.diff(self.p, axis=0) / self.p[:-1]
            self.r = np.concatenate((firstr, self.r), axis=0)

    def set_x_F(self, train_phase=True):
        if train_phase:
            for i in range(self.T - 1, -1, -1):
                self.x[i] = np.zeros(self.M + 2)
                self.x[i][0] = 1.0
                self.x[i][self.M + 2 - 1] = self.F[i+1,-1] ## TODO: i used -1 on column
                for j in range(1, self.M + 2 - 1, 1):
                    #self.x[i][j] = self.r[i+ j - 1,0] ## TODO: i used -1 on column:
                    self.x[i,j] = self.r[i + (j-1), -1]  ## TODO: i used -1 on column; and must deal with j
                self.F[i] = self.quant(np.tanh(np.dot(self.x[i], self.w)+self.b[i]))   ## TODO: test this
        else:
            thisw = np.ones((self.thisM+2, self.N))
            self.x = np.zeros([self.thisT, self.thisM + 2])
            self.F = np.zeros((self.thisT + 1, self.N))
            for i in range(self.thisT - 1, -1, -1):
                self.x[i] = np.zeros(self.thisM + 2)
                self.x[i][0] = 1.0
                self.x[i][self.thisM + 2 - 1] = self.F[i+1,-1] ## TODO: i used -1 on column
                for j in range(1, self.thisM + 2 - 1, 1):
                    #self.x[i][j] = self.r[i+ j - 1,0] ## TODO: i used -1 on column:
                    self.x[i,j] = self.r[i + (j-1), -1]  ## TODO: i used -1 on column; and must deal with j
                self.F[i] = self.quant(np.tanh(np.dot(self.x[i], thisw)+self.b[i]))   ## TODO: test this

    def calc_R(self):
        #self.R = self.mu * (np.dot(self.r[:self.T], self.F[:,1:]) - self.sigma * np.abs(-np.diff(self.F, axis=1)))
        #self.R = self.mu * (self.r[:self.T] * self.F[1:]) - self.sigma * np.abs(-np.diff(self.F, axis=0))
        #self.R = self.mu * (np.multiply(self.F[1:,], np.reshape(self.r[:self.T], (self.T, -1)))) * (self.sigma) * np.abs(-np.diff(self.F, axis=0))
        self.R = ((np.multiply(self.F[1:, ], np.reshape(0+self.r[:self.T], (self.T, -1)))) * (1-self.sigma * np.abs(-np.diff(self.F, axis=0))))
        pd.DataFrame(self.R).to_csv('R.csv')

    def calc_sumR(self):
        self.sumR = np.cumsum(self.R[::-1], axis=0)[::-1] ## TODO: cumsum axis
        #self.sumR = np.cumprod(self.R[::-1], axis=0)[::-1]  ## TODO: cumsum axis
        self.sumR2 = np.cumsum((self.R[::-1] ** 2), axis=0)[::-1] ## TODO: cumsum axis
        #self.sumR2 = np.cumprod((self.R[::-1] ** 2), axis=0)[::-1]  ## TODO: cumsum axis
        #print('cumprod', self.sumR)

    def calc_dSdw(self, train_phase=True):
        if not train_phase:
            self.T = self.thisT
            self.M = self.thisM
        self.set_x_F(train_phase=train_phase)
        self.calc_R()
        self.calc_sumR()

        self.Sall = np.empty(0)  # a list of period-to-date sharpe ratios, for all n investments
        self.dSdw = np.zeros((self.M + 2, self.N))
        for j in range(self.N):
            self.A = self.sumR[0,j] / self.T
            self.B = self.sumR2[0,j] / self.T
            #self.A = self.sumR / self.T
            #self.B = self.sumR2 / self.T
            self.S = self.A / np.sqrt(self.B - (self.A ** 2))
            #self.S = ((self.B[1:,j]*np.diff(self.A[:,j], axis=0)-0.5*self.A[1:,j]*np.diff(self.B[:,j], axis=0))/ (self.B[1,j] - (self.A[1,j] ** 2))**(3/2))[1]
            #self.S = (self.B[1,j] - (self.A[1,j] ** 2))**(3/2)
            #print('sharpe checl', np.isnan(self.r).sum())
            self.dSdA = self.S * (1 + self.S ** 2) / self.A
            self.dSdB = -self.S ** 3 / 2 / self.A ** 2
            self.dAdR = 1.0 / self.T
            self.dBdR = 2.0 / self.T * self.R[:,j]
            self.dRdF = -self.mu * self.sigma * np.sign(-np.diff(self.F, axis=0))
            self.dRdFp = self.mu * self.r[:self.T] + self.mu * self.sigma * np.sign(-np.diff(self.F, axis=0))  ## TODO: r needs to be a matrix if assets > 1
            self.dFdw = np.zeros(self.M + 2)
            self.dFpdw = np.zeros(self.M + 2)
            #self.dSdw = np.zeros((self.M + 2, self.N))  ## TODO: should not have put this here. this resets everytime
            self.dSdw_j = np.zeros(self.M + 2)
            for i in range(self.T - 1, -1, -1):
                if i != self.T - 1:
                    self.dFpdw = self.dFdw.copy()
                self.dFdw = (1 - self.F[i,j] ** 2) * (self.x[i] + self.w[self.M + 2 - 1,j] * self.dFpdw)
                self.dSdw_j += (self.dSdA * self.dAdR + self.dSdB * self.dBdR[i]) * (
                    self.dRdF[i,j] * self.dFdw + self.dRdFp[i,j] * self.dFpdw)
            self.dSdw[:, j] = self.dSdw_j
            self.Sall = np.append(self.Sall, self.S)

    def update_w(self):
        self.w += self.rho * self.dSdw

    def get_investment_weights(self, train_phase=True):
        if not train_phase:
            self.FS = np.zeros((self.thisT + 1, self.N))

        for i in range(self.FS.shape[0]):
            self.FS[i] = np.multiply(self.F[i], self.Sall)
        tmp = np.apply_along_axis(self.select_n, 1, self.FS) # TODO: conisder taking the abs(): magnitutde
        F1 = np.apply_along_axis(self.softmax, 1, tmp)
        print('MAKE F1', F1.shape)
        print('see F1', F1)
        print('see R', self.R)
        mask = F1 != 0
        _, j = np.where(mask)
        for ji in set(j):
            self.ticker_data[self.tickers_list[ji]]['inv weight'] = F1[-2, ji]
            self.ticker_data[self.tickers_list[ji]]['return'] = self.R[-2, ji]
        print(self.ticker_data)
        return F1

    def select_n(self, array):
        threshold = max(heapq.nlargest(self.TOP, array)[-1], self.threshold)
        new_array = [x if x >= threshold else 0 for x in array]
        return new_array

    def fit(self):
        pre_epoch_times = len(self.epoch_S)

        self.calc_dSdw()
        print("Epoch loop start. Initial sharp's ratio is " + str(np.mean(self.Sall)) + ".")
        print('s len', len(self.Sall))
        self.S_opt = self.Sall

        tic = time.clock()
        for e_index in range(self.n_epoch):
            self.calc_dSdw()
            if np.mean(self.Sall) > np.mean(self.S_opt):
                self.S_opt = self.Sall
                self.w_opt = self.w.copy()
            #self.Sall = np.apply_along_axis(self.select_n, 0, self.Sall) # TODO: don't do this here
            self.epoch_S[e_index] = np.array(self.S_opt)
            self.update_w()
            if e_index % self.progress_period == self.progress_period - 1:
                toc = time.clock()
                print("Epoch: " + str(e_index + pre_epoch_times + 1) + "/" + str(
                    self.n_epoch + pre_epoch_times) + ". Shape's ratio: " + str(self.Sall[self.Sall.nonzero()].mean()) + ". Elapsed time: " + str(
                    toc - tic) + " sec.")
        toc = time.clock()
        print("Epoch: " + str(e_index + pre_epoch_times + 1) + "/" + str(
            self.n_epoch + pre_epoch_times) + ". Shape's ratio after iteration: " + str(self.S_opt[self.S_opt.nonzero()].mean()) + ". Elapsed time: " + str(
            toc - tic) + " sec.")
        self.w = self.w_opt.copy()
        self.calc_dSdw()
        print("Epoch loop end. Optimized sharp's ratio is " + str(self.S_opt[self.S_opt.nonzero()].mean()) + ".")
        print('first check', self.Sall)
        print('now check', self.epoch_S)
        print('R dimension', self.R.shape)


    def save_weight(self, train_phase=True):
        if train_phase:
            self.F1 = self.get_investment_weights()

            pd.DataFrame(self.w).to_csv("w.csv", header=False, index=False)
            self.epoch_S.to_csv("epoch_S.csv", header=False, index=False)
            pd.DataFrame(self.F).to_csv("f.csv", header=False, index=False)
            pd.DataFrame(self.FS).to_csv("fs.csv", header=False, index=False)
            pd.DataFrame(self.F1).to_csv("f1.csv", header=False, index=False)
        else:
            self.F1 = self.get_investment_weights(train_phase=False)
            pd.DataFrame().from_dict(self.ticker_data).T.to_csv('ticker_data.csv')

    def load_weight(self):
        tmp = pd.read_csv("w.csv", header=None)
        self.w = tmp.T.values[0]

    def get_investment_sum(self, train_phase=True):
        firstR = np.zeros((1,self.p.shape[1]))
        self.R = np.concatenate((firstR, self.R), axis=0)
        tmp = np.multiply(self.R, self.F1)
        self.total = self.mu * ((1+tmp.sum(axis=1)).cumprod(axis=0))
        print('iam here', self.total.shape, self.total)
        if train_phase:
            pd.DataFrame(self.total).to_csv('investment_sum.csv')
        else:
            pd.DataFrame(self.total).to_csv('investment_sum_testphase.csv')